In [61]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

#### Resizing Model Embeddings (50527) to Match with Tokenizer Vocabulary Size (50528)

In [6]:
'''
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

model_path = "gpt2-small/checkpoint-trainedtokenizer_100M"

tokenizer = GPT2TokenizerFast.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# print mismatch
print("Tokenizer vocab size:", len(tokenizer))
print("Model vocab size:", model.config.vocab_size)

# resize model embeddings to match tokenizer
if len(tokenizer) != model.config.vocab_size:
    print(f"Resizing model embeddings from {model.config.vocab_size} → {len(tokenizer)}")
    model.resize_token_embeddings(len(tokenizer))
    model.save_pretrained(model_path)
    print("Saved updated model.")
'''

Tokenizer vocab size: 50258
Model vocab size: 50257
Resizing model embeddings from 50257 → 50258
Saved updated model.
Special tokens map: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}


In [39]:
model_path = "gpt2-small/checkpoint-trainedtokenizer_100M"

model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

# wrap with minicons scorer
lm_scorer = scorer.IncrementalLMScorer(model_path, device = "cpu")

In [63]:
print("Special tokens:", tokenizer.all_special_tokens)
print("Special token IDs:", tokenizer.all_special_ids)

Special tokens: ['<|endoftext|>']
Special token IDs: [50257]


In [40]:
print("Special tokens map:", tokenizer.special_tokens_map)

Special tokens map: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}


In [17]:
surprisal

[[('<pad>', 0.0),
  ('ĠTheĠ', 4.693825721740723),
  ('balloon', 9.007822036743164),
  ('Ġwa', 2.1047677993774414),
  ('s', 0.002323150634765625),
  ('Ġinf', 10.309831619262695),
  ('lat', 2.666797637939453),
  ('ingĠfor', 6.27564811706543),
  ('Ġ10', 7.321373462677002),
  ('Ġminute', 2.1019039154052734),
  ('s', 0.0007076263427734375)],
 [('<pad>', 0.0),
  ('Ġpl', 7.494743824005127),
  ('ace', 5.744235992431641),
  ('holder', 11.565694808959961)]]

In [72]:
# takes in a sentence, and output the surprisal values for each word

def calculate_surprisal(sentence):
    input_sentence = sentence # process per sentence, never in batches to avoid padding
    # token_score() function of Minicons takes in several parameters
    # if surprisal = True, the output value is surprisal instead of log likelihood
    # if base_two = True, the log likelihood will be in base 2
    # see Minicons documentations for details
    # score tokens
    token_surprisals = lm_scorer.token_score(input_sentence, surprisal = True, base_two = True)[0]

    # tokenizer setup
    encoding = tokenizer(sentence, return_offsets_mapping = True, add_special_tokens = False)
    offsets = encoding['offset_mapping']
    token_ids = encoding['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(token_ids)

    # filter out special token surprisals (like <pad>) *not needed if we input sentences one-by-one, but just to be safe
    special_tokens = set(tokenizer.all_special_tokens + ['<pad>'])
    filtered = [
        (token, score, span)
        for (token, score), span in zip(token_surprisals, offsets)
        if token not in special_tokens
    ]

    # prepare: group surprisals by words based on character spans
    words = re.findall(r"\S+", sentence)
    word_spans = []
    i = 0
    for match in re.finditer(r"\S+", sentence):
        start, end = match.span()
        word_spans.append((i, start, end))
        i += 1

    # assign tokens to words based on character alignment (needed since BPE tokenizers break words down into subwords/tokens)
    word_surprisals = []
    word_index = 0
    word_start, word_end = word_spans[word_index][1:3] # previously: word_spans.append((i, start, end)) [0, 1, 2]
    current_surprisal = 0.0

    for token, score, (start, end) in filtered:
        if start >= word_end:
            word_surprisals.append((words[word_index], current_surprisal))
            word_index += 1
            if word_index >= len(word_spans):
                break
            word_start, word_end = word_spans[word_index][1:3]
            current_surprisal = 0.0
        current_surprisal += score

    # append final word
    if word_index < len(words):
        word_surprisals.append((words[word_index], current_surprisal))

    return word_surprisals

sentence = 'The balloon was inflating for 10 minutes'
calculate_surprisal(sentence)

[('The', 0.0),
 ('balloon', 4.693825721740723),
 ('was', 11.112587928771973),
 ('inflating', 12.978955268859863),
 ('for', 6.275647163391113),
 ('10', 7.32137393951416),
 ('minutes', 2.101907730102539)]

#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [ ]:
# columns: item_id (indicating sentence set), wh_licensor (0/1), gap (0/1), island_presence (0 = non-island, 1 = island), island_type, surprisal
sentence_df = pd.read_csv('test_sentences.csv')
sentence_df['wh_licensor'] = sentence_df['wh_licensor'].astype('category')
sentence_df['gap'] = sentence_df['gap'].astype('category')
sentence_df['island_presence'] = sentence_df['island_presence'].astype('category')
sentence_df['island_type'] = sentence_df['island_type'].astype('category')

# get the list of unique island types (excluding non-islands denoted by 'None')
island_types = [island for island in data['island_type'].unique() if it != 'None']

# list to store results
interaction_results = []

def mixed_effects_linear_regression(subset, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
    model = smf.mixedlm(
        "surprisal ~ wh_licensor * gap",
        subset,
        groups = subset["item_id"]
    )
    result = model.fit()
    interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)
    
    print(f"\n=== {label.upper()} ===")
    print(result.summary())
    
    return interaction_coef

# loop through each island type
for island in island_types:
    # subset data for this island type + corresponding non-island
    subset = sentence_df[(sentence_df['island_type'] == island)]
    
    # check if enough data
    if subset.shape[0] < 8:
        print(f"Not enough data points for {island}. Skipping.")
        continue

    # separate non-island and island subsets within island type
    non_island_subset = subset[subset['island_presence'] == 0]
    island_subset = subset[subset['island_presence'] == 1]
    
    # calculate wh-licensing interaction terms separately
    interaction_non_island = mixed_effects_linear_regression(non_island_subset, f"{island} (non-island)")
    interaction_island = mixed_effects_linear_regression(island_subset, f"{island} (island)")

    # save results
    interaction_results.append({
        "Island Type": island,
        "Condition": "non-island",
        "Wh-Licensing Interaction": interaction_non_island
    })
    interaction_results.append({
        "Island Type": island,
        "Condition": "island",
        "Wh-Licensing Interaction": interaction_island
    })

results_df = pd.DataFrame(interaction_results)
results_df.to_csv('wh_licensing_interactions_by_island_type.csv', index = False)

# plot results
plt.figure(figsize = (10, 6))
sns.barplot(
    data = results_df,
    x = "Island Type",
    y = "Wh-Licensing Interaction",
    hue = "Condition",
    palette = "muted"
)
plt.axhline(0, color = "black", linestyle = "--")
plt.title("Wh-Licensing Interaction Across Different Island Types")
plt.ylabel("Interaction Size")
plt.xticks(rotation = 45)
plt.legend(title = "Sentence Condition")
plt.tight_layout()
plt.show()